<a href="https://colab.research.google.com/github/AmbrogioMB/AlgOpt/blob/main/Linear_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Basics of Gurobi: A simple LP problem
In this section, we define and solve a simple linear programming (LP) problem using Gurobi.
The goal is to maximize the objective function 3x + 5y subject to a set of linear constraints.
We will learn how to define variables, set an objective function, add constraints, and solve the problem.

In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("Simple LP")

# Variables
x = model.addVar(name="x", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)
y = model.addVar(name="y", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)

# Objective
model.setObjective(3*x + 5*y, GRB.MAXIMIZE)

# Constraints
model.addConstr(2*x + y <= 8, "Constraint1")
model.addConstr(x + 2*y <= 6, "Constraint2")
model.addConstr(x >= 1, "Constraint3")
model.addConstr(y >= 2, "Constraint4")

# Optimize
model.optimize()

# Results
if model.status == GRB.OPTIMAL:
    print(f"Optimal value: {model.objVal}")
    print(f"x = {x.x}, y = {y.x}")

## 2. Standard form of LP
In this section, we convert the LP to its standard form by introducing slack variables.
Inequality constraints are transformed into equalities by adding non-negative slack variables.

In [ ]:
model_sf = gp.Model("Standard Form LP")

# Variables
x_sf = model_sf.addVar(name="x", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)
y_sf = model_sf.addVar(name="y", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)
s1 = model_sf.addVar(name="s1", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)
s2 = model_sf.addVar(name="s2", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)
s3 = model_sf.addVar(name="s3", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)
s4 = model_sf.addVar(name="s4", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)

# Objective
model_sf.setObjective(3*x_sf + 5*y_sf, GRB.MAXIMIZE)

# Constraints
model_sf.addConstr(2*x_sf + y_sf + s1 == 8, "StandardForm1")
model_sf.addConstr(x_sf + 2*y_sf + s2 == 6, "StandardForm2")
model_sf.addConstr(x_sf  - s3 == 1, "StandardForm3")
model_sf.addConstr(y_sf - s4 == 2, "StandardForm4")

# Optimize
model_sf.optimize()

# Results
if model_sf.status == GRB.OPTIMAL:
    print(f"Optimal value: {model_sf.objVal}")
    print(f"x = {x_sf.x}, y = {y_sf.x}")
    print(f"Slack variable s1 = {s1.x}, s2 = {s2.x}, s3 = {s3.x}, s4 = {s4.x}")


## 3. Dual of the LP
Here we define the dual of the original LP problem.
The dual problem provides a lower bound for the optimal value of the primal problem.
We define and solve the dual to illustrate the relationship between the primal and dual solutions.


In [ ]:
dual = gp.Model("Dual LP")

u = dual.addVar(name="u", vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)
v = dual.addVar(name="v", vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)
w1 = dual.addVar(name="w1", vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)
w2 = dual.addVar(name="w2", vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)

dual.setObjective(8*u + 6*v + w1 + 2*w2, GRB.MINIMIZE)

dual.addConstr(2*u + v + w1 >= 3, "DualConstraint1")
dual.addConstr(u + 2*v + w2 >= 5, "DualConstraint2")
dual.addConstr(u >= 0, "DualConstraint3")
dual.addConstr(v >= 0, "DualConstraint4")
dual.addConstr(- w1 >= 0, "DualConstraint5")
dual.addConstr(- w2 >= 0, "DualConstraint6")

dual.optimize()

if dual.status == GRB.OPTIMAL:
    print(f"Dual optimal value: {dual.objVal}")
    print(f"u = {u.x}, v = {v.x}, w1 = {w1.x}, w2 = {w2.x}")

## 4. Complementary slackness conditions
Complementary slackness is a key optimality condition linking the primal and dual solutions.
It states that the product of each primal slack and the corresponding dual variable must be zero.
We verify this condition to confirm the optimality of our solutions.

In [ ]:
if model_sf.status == GRB.OPTIMAL and dual.status == GRB.OPTIMAL:
    primal_sol = [x_sf.x, y_sf.x, s1.x, s2.x, s3.x, s4.x]
    dual_slacks = [3 - (2*u.x + v.x + w1.x), 5 - (u.x + 2*v.x + w2.x), 0 - u.x, 0 - v.x, 0 + w1.x, 0 + w2.x]

    print("Primal slack values:", primal_sol)
    print("Dual slack values:", dual_slacks)

    for i, (ps, ds) in enumerate(zip(primal_sol, dual_slacks)):
        print(f"Complementary slackness for constraint {i+1}: {ps * ds}")


## 5. More complex LP using indexed variables and loops
In this part, we model a production optimization problem for a company producing three products (A, B, C).
Each product requires different amounts of two limited resources. The goal is to maximize total profit.
We use indexed variables and construct constraints using loops to handle multiple products and resources efficiently.

In [ ]:
products = ["A", "B", "C"]
profits = {"A": 5, "B": 7, "C": 4}
resources = ["Resource1", "Resource2"]
availability = {"Resource1": 100, "Resource2": 80}

usage = {("A", "Resource1"): 6, ("A", "Resource2"): 4,
         ("B", "Resource1"): 10, ("B", "Resource2"): 5,
         ("C", "Resource1"): 4, ("C", "Resource2"): 7}

model_complex = gp.Model("Complex LP")

# Indexed variables
produce = model_complex.addVars(products, name="produce", vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY)

# Objective
model_complex.setObjective(gp.quicksum(profits[p] * produce[p] for p in products), GRB.MAXIMIZE)

# Resource constraints using loops
for r in resources:
    model_complex.addConstr(gp.quicksum(usage[p, r] * produce[p] for p in products) <= availability[r], name=f"Resource_{r}")

# Optimize
model_complex.optimize()

if model_complex.status == GRB.OPTIMAL:
    print(f"Optimal profit: {model_complex.objVal}")
    for p in products:
        print(f"Production of {p}: {produce[p].x}")

## 6. Exercise: Define and solve a warehouse optimization LP
 A company manages the distribution of goods from 3 warehouses to 4 stores.
 Transport costs and supply/demand are known. Formulate and solve an LP to minimize shipping costs.

 Variables: Amount shipped from each warehouse to each store.

 Objective: Minimize total shipping costs.

 Constraints:
 - Shipments from each warehouse must not exceed its supply.
 - Shipments to each store must meet its demand.

In [ ]:
warehouses = ["W1", "W2", "W3"]
stores = ["S1", "S2", "S3", "S4"]

supply = {"W1": 100, "W2": 150, "W3": 120}
demand = {"S1": 80, "S2": 90, "S3": 100, "S4": 100}
cost = {("W1", "S1"): 4, ("W1", "S2"): 6, ("W1", "S3"): 8, ("W1", "S4"): 10,
        ("W2", "S1"): 3, ("W2", "S2"): 5, ("W2", "S3"): 6, ("W2", "S4"): 8,
        ("W3", "S1"): 7, ("W3", "S2"): 8, ("W3", "S3"): 5, ("W3", "S4"): 6}

## 7. Constraint matrix

We will now see an easy way to get the constrain matrix of an LP such that it is written as $A'x \leq b'$.

In [ ]:
import numpy as np
A = model.getA()
b_new = []
A_new = []

# Process each constraint
for i, constr in enumerate(model.getConstrs()):
    # Convert the sparse row to dense (numpy array)
    row = A.getrow(i).todense()  # or A[i,:].A to convert sparse to dense
    rhs = constr.RHS
    s = constr.Sense

    if s == '<':
        A_new.append(row)
        b_new.append(rhs)

    elif s == '>':
        A_new.append(-row)
        b_new.append(-rhs)

    elif s == '=':
        # Add two inequalities for equality constraints
        A_new.append(row)
        b_new.append(rhs)
        A_new.append(-row)
        b_new.append(-rhs)

# Stack into final A' and b'
A_final = np.vstack(A_new)  # Stack rows
b_final = np.array(b_new)   # Stack the right-hand side values

# Optional: Convert to dense for inspection (although it's already dense now)
print("A':")
print(A_final)

print("\nb':")
print(b_final)